## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='random-forest-regressor', tags={}>,
 <Experiment: artifact_location='artifacts/3', experiment_id='3', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>]

In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

In [4]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-cool-experiment', tags={}>]

Let's check the latest versions for the experiment with id `1`...

In [3]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='3',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [4]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: e3a6c169dd9341b3903607220cc2411c, rmse: 6.6284
run id: b6014567a4bb4ab49fe96634ef93956a, rmse: 6.6297
run id: 6eadf84318144ef884f29166083a9a63, rmse: 6.6299
run id: aba18fcf1e3442ae8df9ec4df85281b1, rmse: 6.6299
run id: dbb35df8daf14e20a28d0693024a5abb, rmse: 6.6299


In [7]:
import mlflow
best_run = runs[0]

# register the best model
best_run_uri = f"runs:/{best_run.info.run_id}/model"
mlflow.register_model(best_run_uri, "best-model")

MlflowException: Model Registry features are not supported by the store with URI: 'file:///D:/Document/Personal-Codes/14-MLOps-DataTalksClub/mlops-zoomcamp/personal-notebooks/02-experiment-tracking/mlruns'. Stores with the following URI schemes are supported: ['databricks', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql'].

In [11]:
type(runs[0].info.run_id)

str

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [12]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/26 21:24:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653575055633, current_stage='None', description=None, last_updated_timestamp=1653575055633, name='nyc-taxi-regressor', run_id='bb1a2740440a444a8231d91d5f027e92', run_link=None, source='./mlruns/1/bb1a2740440a444a8231d91d5f027e92/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [31]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Production
version: 3, stage: Staging


In [29]:
# Stage version 2 into Production
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

client.update_model_version(
    name=model_name,
    version=2,
    description=f"The model version 3 was transitioned to Production"
)

<ModelVersion: creation_timestamp=1653574346365, current_stage='Production', description='The model version 3 was transitioned to Production', last_updated_timestamp=1653579861226, name='nyc-taxi-regressor', run_id='3f72ed3d9d9f4a8f8c7bf3d1c0415996', run_link='', source='./mlruns/1/3f72ed3d9d9f4a8f8c7bf3d1c0415996/artifacts/model', status='READY', status_message=None, tags={'model': 'GradientBoostingRegressor'}, user_id=None, version=2>

In [30]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653575055633, current_stage='Staging', description='The model version 3 was transitioned to Production', last_updated_timestamp=1653579863480, name='nyc-taxi-regressor', run_id='bb1a2740440a444a8231d91d5f027e92', run_link=None, source='./mlruns/1/bb1a2740440a444a8231d91d5f027e92/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [16]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653575055633, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-05-26', last_updated_timestamp=1653577738830, name='nyc-taxi-regressor', run_id='bb1a2740440a444a8231d91d5f027e92', run_link=None, source='./mlruns/1/bb1a2740440a444a8231d91d5f027e92/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [17]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
    
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [18]:
df = read_dataframe("../data/green_tripdata_2021-03.parquet")

In [20]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='./models')

'/Users/husni.zuhdi/Documents/Codes/1-Python-Workshop/mlops-zoomcamp/personal-notebooks/02-experiment-tracking/models/preprocessor'

In [22]:
import pickle

with open("models/preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [23]:
X_test = preprocess(df, dv)

In [24]:
target = "duration"
y_test = df[target].values

In [32]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 328 ms, sys: 103 ms, total: 431 ms
Wall time: 590 ms


{'rmse': 6.659623830022514}

In [34]:
%time test_model(name=model_name, stage="Archived", X_test=X_test, y_test=y_test)

CPU times: user 7.88 s, sys: 718 ms, total: 8.6 s
Wall time: 8.99 s


{'rmse': 6.886564762449945}